In [12]:
import platform
print(f"{platform.architecture()[0]} compatible with CPLEX/GLPK/SCIP")

64bit compatible with CPLEX/GLPK/SCIP


In [13]:
import pandas as pd
import numpy as np
try:
    from pyomo.environ import *
    print(f"Pyomo Version : {pyomo.__version__}")
except IOError:
    raise ImportError("Pyomo not available")

Pyomo Version : 6.6.2


In [14]:
import pandas as pd
import numpy as np
from pyomo.opt import SolverStatus,TerminationCondition
import logging
from pyomo.util.infeasible import log_infeasible_constraints

In [15]:
class VRPTWDeco(ConcreteModel,metaclass=type):
    
    def __init__(self,data:pd.DataFrame, vehicle_max_capacity:int=200, instance_name:str="VRPTW_C101"):
        
        ConcreteModel.__init__(self,instance_name)
        
              
        self.data = data
        
        self.customers = [i for i in range(1,len(self.data))]
        print(f"Number of Customers :: {len(self.customers)}")
        
        
        self.vertex = [0] + self.customers
        self.arc1 = [(i,j) for i in self.vertex for j in self.vertex if i!=j]
        
        self.c = {}
        self.c = self.distance_matrix()
        self.vehicle_max_capacity = vehicle_max_capacity
        self.ready_time = self.data["READY_TIME"].to_list()
        self.due_date = self.data["DUE_DATE"].to_list()
        
        self.arc = Set(initialize=self.arc1)
        self.dist = Param(self.arc,initialize=self.c)
        self.max_cap = Param(initialize=self.vehicle_max_capacity)
        self.cust = Set(initialize=self.customers)
        self.x = Var(self.arc,within=Binary)
        self.u = Var(self.cust,within=Reals,bounds=self._u_bounds)
        self.time2 = Var(self.vertex,bounds=self._time2_bounds)
        self.depart = Var(self.vertex,within=NonNegativeReals)
        #self.model.objective = Objective()
        self.status = None
        #self.solver = None
        
        #You need to write objective function and constraints within init() otherwise it will give error "self" not defined
        '''Objective Function'''
        @self.Objective()
        def obj(s):
            return sum(s.c[(i,j)]*s.x[(i,j)] for i,j in s.arc)
        
        '''Constraint 1'''
        @self.Constraint(self.cust)
        def _constraint1(s,i):
            lhs = sum(s.x[i,j] for j in s.vertex if i!=j)
            rhs=1
            return lhs==rhs
        
        '''Constraint 2'''
        @self.Constraint(self.cust)
        def _constraint2(s,j):
            lhs= sum(s.x[i,j] for i in s.vertex if i!=j)
            rhs=1
            return lhs==rhs
        
        '''constraint 3'''
        @self.Constraint(self.arc)
        def _constraint3(s,i,j):
                if i != 0 and j!=0:
                     return  s.u[i] + s.data["DEMAND"].iloc[j] - (1-s.x[i,j])*99999 <= s.u[j]
                else:
                     return Constraint.Skip
                     
            
        
        '''Constraint 4'''
        @self.Constraint(self.cust)
        def _constraint4(s,i):
            lhs = s.time2[i] + s.data["SERVICE_TIME"].iloc[i]
            rhs = s.depart[i]
            return lhs <= rhs
        
        '''Constraint 5'''
        @self.Constraint(self.arc)
        def _constraint5(s,i,j):
            if i != j and j != 0:
                 lhs = s.time2[i] + s.data["SERVICE_TIME"].iloc[i] + s.c[i,j] - (1-s.x[i,j])*99999
                 rhs = s.time2[j]
                 return lhs <= rhs
            else:
                return Constraint.Skip
      
    def _distance(self,i,j):
        self.cust1_x = self.data["XCOORD"].iloc[i]
        self.cust1_y = self.data["YCOORD"].iloc[i]
        self.cust2_x = self.data["XCOORD"].iloc[j]
        self.cust2_y = self.data["YCOORD"].iloc[j]
        return round(np.sqrt((self.cust1_x-self.cust2_x)**2+(self.cust1_y-self.cust2_y)**2),2)
    
    def distance_matrix(self):
        for i,j in self.arc1:
            self.c[(i,j)] = self._distance(i,j)
        return self.c
    
    def _u_bounds(self,model):
        return (0,self.max_cap)
    
     
    def _time2_bounds(self,model,i):
         return (self.ready_time[i],self.due_date[i])   
        
                        
    def _check_math_model(self,verbose=True):
        if verbose:
            print(self.objective.expr)
            print("\n\n")
            print("s.t")
            print(self.con1[1].expr)
            print("\n")
            print(self.con2[1].expr)
            print("\n")
            print(self.con3[1].expr)
            print("\n")
            print(self.con4[1].expr)
            print("\n")
            print(self.con5[1].expr)
    
    def solve(self):
        self.solver = SolverFactory("scip",executable="C:/Program Files/SCIPOptSuite 8.0.1/bin/scip.exe")
        self.solver.options["timelimit"] = 60 #in seconds
        self.solver.options["limit/gap"] = 0.1 # 10%
        self.results = self.solver.solve(self,tee=True)
        
        '''Write LP File'''
        self.write("D:/VRPTW_DECORATOR_Inheritance.lp")
        
        if (self.results.solver.status == SolverStatus.ok) and (self.results.solver.termination_condition == TerminationCondition.optimal):
            print("\n\n Optimal Solution")
        elif (self.results.solver.termination_condition == TerminationCondition_infeasible):
            '''Check for Infeasibility'''
            print("\n\n Infeasible Solution")
            log_infeasible_constraints(self, log_expression =True, log_variables=True)
            logging.basicConfig(filename="D:/VRPTW_infeasibility.log",encoding ='utf-8',level = logging.INFO)
        else:
            print(f"\n\n Solver Status : {self.result.solver.status}")
        for i,j in self.arc:
             if value(self.x[i,j]) > 0:
                print(f"x[{i},{j}]=1")
        print(f"Distance Traveresed : {value(self.obj())}")
                      
                
        
        

In [25]:
#load data from Homberger 200 customer's instance
PATH = r"D:\Homberger_200_customers\homberger_200_customer_instances\c1_2_1.txt"
instance = None
vehicle_number=0
vehicle_cap=0.0
col_name=["CUST_NO","XCOORD","YCOORD","DEMAND","READY_TIME","DUE_DATE","SERVICE_TIME"]

#Method -1
#cust_no=[]
#xcoord=[]
#ycoord=[]
#demand=[]
#ready_time=[]
#due_date=[]
#service_time=[]

#Method-2
data = []
with open(PATH,"r") as f:
    content = f.readlines()
    for ind, line in enumerate(content):
        if ind==0:
            instance=line
            print(f"Instance Name: {instance}")
        if ind==4:
            val = line.split(" ")
            val = list(filter(str.strip,val))
            #print(val)
            vehicle_number = int(val[0])
            vehicle_cap = float(val[1])
            print(f"Vehicle Number {vehicle_number} and Vehicle Capacity {vehicle_cap}")
        if ind >= 9:
            val = line.split(" ")
            val = [x for x in val if x]
            #Method-2
            data.append((int(val[0]),float(val[1]),float(val[2]),float(val[3]),float(val[4]),float(val[5]),float(val[6])))
            
            #Method-1
            #cust_no.append(int(val[0]))
            #xcoord.append(float(val[1]))
            #ycoord.append(float(val[2]))
            #demand.append(float(val[3]))
            #ready_time.append(float(val[4]))
            #due_date.append(float(val[5]))
            #service_time.append(float(val[6]))

#Method-1            
#data = pd.DataFrame({"CUST_NO":cust_no,"XCOORD":xcoord,"YCOORD":ycoord,"DEMAND":demand,"READY_TIME":ready_time,"DUE_DATE":due_date,"SERVICE_TIME":service_time})
#Method-2
data = pd.DataFrame(data,columns=col_name)
data.head()

Instance Name: c1_2_1

Vehicle Number 50 and Vehicle Capacity 200.0


,CUST_NO,XCOORD,YCOORD,DEMAND,READY_TIME,DUE_DATE,SERVICE_TIME
0,0,70.0,70.0,0.0,0.0,1351.0,0.0
1,1,33.0,78.0,20.0,750.0,809.0,90.0
2,2,59.0,52.0,20.0,553.0,602.0,90.0
3,3,10.0,137.0,30.0,147.0,219.0,90.0
4,4,4.0,28.0,10.0,616.0,661.0,90.0


In [26]:
vrptw = VRPTWDeco(data=data)
vrptw.solve()


Number of Customers :: 200
SCIP version 8.0.1 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex 6.0.1] [GitHash: c84ee4283e]
Copyright (C) 2002-2022 Konrad-Zuse-Zentrum fuer Informationstechnik Berlin (ZIB)

External libraries: 
  Soplex 6.0.1         Linear Programming Solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: 8b86b300]
  CppAD 20180000.0     Algorithmic Differentiation of C++ algorithms developed by B. Bell (github.com/coin-or/CppAD)
  MPIR 3.0.0           Multiple Precision Integers and Rationals Library developed by W. Hart (mpir.org)
  ZIMPL 3.5.2          Zuse Institute Mathematical Programming Language developed by T. Koch (zimpl.zib.de)
  AMPL/MP 4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp)
  PaPILO 2.1.0         parallel presolve for integer and linear optimization (github.com/scipopt/papilo) [GitHash: 9363218]
  bliss 0.77           Computing Graph Automorphism Groups by T. Junttila and P. Kaski (www.tcs.hut.f

In [27]:
print(f"Distance Traveresed : {value(vrptw.obj())}")

Distance Traveresed : 2700.3599999999965
